In [5]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WikipediaNetwork
from torch_geometric.loader import DataLoader

In [2]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

10920
7290
4560


# GCN using only domain Feature

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 48, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-5, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0705, Train: 24.91%, Valid: 21.67% Test: 23.90%
Run: 01, Epoch: 02, Loss: 1.3888, Train: 34.89%, Valid: 29.77% Test: 31.80%
Run: 01, Epoch: 03, Loss: 1.3086, Train: 35.16%, Valid: 32.24% Test: 32.24%
Run: 01, Epoch: 04, Loss: 1.2730, Train: 34.98%, Valid: 30.73% Test: 31.36%
Run: 01, Epoch: 05, Loss: 1.2449, Train: 39.74%, Valid: 35.53% Test: 33.99%
Run: 01, Epoch: 06, Loss: 1.2176, Train: 44.78%, Valid: 37.31% Test: 39.25%
Run: 01, Epoch: 07, Loss: 1.2017, Train: 46.61%, Valid: 40.19% Test: 41.01%
Run: 01, Epoch: 08, Loss: 1.1655, Train: 49.82%, Valid: 43.62% Test: 43.42%
Run: 01, Epoch: 09, Loss: 1.1692, Train: 52.66%, Valid: 45.82% Test: 47.15%
Run: 01, Epoch: 10, Loss: 1.1402, Train: 53.94%, Valid: 48.83% Test: 48.46%
Run: 01, Epoch: 11, Loss: 1.1236, Train: 57.69%, Valid: 52.13% Test: 51.97%
Run: 01, Epoch: 12, Loss: 1.1029, Train: 60.26%, Valid: 55.01% Test: 54.39%
Run: 01, Epoch: 13, Loss: 1.0898, Train: 60.99%, Valid: 55.97% Test: 55.48%
Run: 01, Epo

Run: 02, Epoch: 07, Loss: 1.2599, Train: 37.27%, Valid: 33.06% Test: 36.62%
Run: 02, Epoch: 08, Loss: 1.2285, Train: 38.00%, Valid: 34.02% Test: 37.28%
Run: 02, Epoch: 09, Loss: 1.2054, Train: 39.38%, Valid: 34.29% Test: 39.04%
Run: 02, Epoch: 10, Loss: 1.1993, Train: 45.05%, Valid: 39.23% Test: 44.52%
Run: 02, Epoch: 11, Loss: 1.1768, Train: 46.43%, Valid: 39.92% Test: 45.61%
Run: 02, Epoch: 12, Loss: 1.1753, Train: 49.54%, Valid: 42.52% Test: 47.15%
Run: 02, Epoch: 13, Loss: 1.1534, Train: 52.93%, Valid: 43.90% Test: 49.78%
Run: 02, Epoch: 14, Loss: 1.1305, Train: 55.22%, Valid: 46.78% Test: 51.10%
Run: 02, Epoch: 15, Loss: 1.1274, Train: 58.79%, Valid: 50.21% Test: 53.73%
Run: 02, Epoch: 16, Loss: 1.1116, Train: 61.54%, Valid: 50.21% Test: 54.17%
Run: 02, Epoch: 17, Loss: 1.0881, Train: 62.82%, Valid: 50.75% Test: 55.48%
Run: 02, Epoch: 18, Loss: 1.0880, Train: 64.38%, Valid: 51.44% Test: 57.24%
Run: 02, Epoch: 19, Loss: 1.0726, Train: 64.19%, Valid: 51.99% Test: 57.46%
Run: 02, Epo

Run: 03, Epoch: 23, Loss: 1.0348, Train: 61.72%, Valid: 52.13% Test: 50.66%
Run: 03, Epoch: 24, Loss: 1.0251, Train: 61.36%, Valid: 51.58% Test: 50.00%
Run: 03, Epoch: 25, Loss: 1.0204, Train: 62.36%, Valid: 52.13% Test: 49.12%
Run: 03, Epoch: 26, Loss: 0.9987, Train: 63.00%, Valid: 52.13% Test: 49.34%
Run: 03, Epoch: 27, Loss: 1.0006, Train: 63.28%, Valid: 52.67% Test: 49.34%
Run: 03, Epoch: 28, Loss: 0.9987, Train: 63.74%, Valid: 52.81% Test: 49.34%
Run: 03, Epoch: 29, Loss: 0.9685, Train: 63.92%, Valid: 52.81% Test: 49.12%
Run: 03, Epoch: 30, Loss: 0.9599, Train: 64.84%, Valid: 55.56% Test: 51.10%
Run: 03, Epoch: 31, Loss: 0.9487, Train: 65.66%, Valid: 56.79% Test: 51.97%
Run: 03, Epoch: 32, Loss: 0.9453, Train: 66.03%, Valid: 56.79% Test: 52.41%
Run: 03, Epoch: 33, Loss: 0.9319, Train: 66.58%, Valid: 56.93% Test: 52.41%
Run: 03, Epoch: 34, Loss: 0.9014, Train: 67.58%, Valid: 56.93% Test: 51.97%
Run: 03, Epoch: 35, Loss: 0.8913, Train: 68.32%, Valid: 56.79% Test: 51.97%
Run: 03, Epo

Run: 04, Epoch: 31, Loss: 0.8646, Train: 69.32%, Valid: 59.40% Test: 57.68%
Run: 04, Epoch: 32, Loss: 0.8700, Train: 69.69%, Valid: 58.44% Test: 58.99%
Run: 04, Epoch: 33, Loss: 0.8444, Train: 70.97%, Valid: 57.61% Test: 59.65%
Run: 04, Epoch: 34, Loss: 0.8403, Train: 71.34%, Valid: 57.61% Test: 60.31%
Run: 04, Epoch: 35, Loss: 0.8310, Train: 72.99%, Valid: 58.71% Test: 60.09%
Run: 04, Epoch: 36, Loss: 0.7960, Train: 73.26%, Valid: 60.22% Test: 58.99%
Run: 04, Epoch: 37, Loss: 0.8033, Train: 72.99%, Valid: 61.32% Test: 59.21%
Run: 04, Epoch: 38, Loss: 0.7875, Train: 73.99%, Valid: 61.32% Test: 59.43%
Run: 04, Epoch: 39, Loss: 0.7840, Train: 74.91%, Valid: 61.87% Test: 59.87%
Run: 04, Epoch: 40, Loss: 0.7485, Train: 76.56%, Valid: 62.55% Test: 59.87%
Run: 04, Epoch: 41, Loss: 0.7649, Train: 74.82%, Valid: 60.49% Test: 59.43%
Run: 04, Epoch: 42, Loss: 0.7460, Train: 75.55%, Valid: 60.22% Test: 59.65%
Run: 04, Epoch: 43, Loss: 0.7313, Train: 76.19%, Valid: 61.04% Test: 59.65%
Run: 04, Epo

Run: 05, Epoch: 43, Loss: 0.6298, Train: 79.58%, Valid: 64.47% Test: 63.16%
Run: 05, Epoch: 44, Loss: 0.6291, Train: 80.49%, Valid: 64.06% Test: 62.94%
Run: 05, Epoch: 45, Loss: 0.6219, Train: 80.31%, Valid: 63.92% Test: 64.47%
Run: 05, Epoch: 46, Loss: 0.6159, Train: 81.59%, Valid: 64.20% Test: 64.91%
Run: 05, Epoch: 47, Loss: 0.5981, Train: 81.96%, Valid: 63.92% Test: 64.91%
Run: 05, Epoch: 48, Loss: 0.5982, Train: 81.50%, Valid: 63.65% Test: 63.60%
Run: 05, Epoch: 49, Loss: 0.5828, Train: 81.14%, Valid: 62.55% Test: 64.04%
Run: 05, Epoch: 50, Loss: 0.5736, Train: 81.87%, Valid: 61.59% Test: 64.25%
Run: 05, Epoch: 51, Loss: 0.5542, Train: 82.05%, Valid: 62.83% Test: 63.82%
Run: 05, Epoch: 52, Loss: 0.5544, Train: 83.06%, Valid: 63.92% Test: 65.35%
Run: 05, Epoch: 53, Loss: 0.5428, Train: 83.61%, Valid: 63.79% Test: 65.57%
Run: 05, Epoch: 54, Loss: 0.5336, Train: 83.79%, Valid: 63.65% Test: 64.69%
Run: 05, Epoch: 55, Loss: 0.5186, Train: 83.88%, Valid: 63.79% Test: 64.25%
Run: 05, Epo

Run: 06, Epoch: 62, Loss: 0.5575, Train: 83.06%, Valid: 64.61% Test: 63.38%
Run: 06, Epoch: 63, Loss: 0.5511, Train: 83.52%, Valid: 64.47% Test: 63.38%
Run: 06, Epoch: 64, Loss: 0.5461, Train: 83.33%, Valid: 64.75% Test: 63.82%
Run: 06, Epoch: 65, Loss: 0.5447, Train: 83.24%, Valid: 64.06% Test: 64.91%
Run: 06, Epoch: 66, Loss: 0.5258, Train: 84.16%, Valid: 65.43% Test: 64.25%
Run: 06, Epoch: 67, Loss: 0.5110, Train: 84.89%, Valid: 65.84% Test: 64.25%
Run: 06, Epoch: 68, Loss: 0.5085, Train: 85.26%, Valid: 66.94% Test: 64.69%
Run: 06, Epoch: 69, Loss: 0.5027, Train: 84.62%, Valid: 65.98% Test: 64.69%
Run: 06, Epoch: 70, Loss: 0.5010, Train: 83.97%, Valid: 65.29% Test: 64.04%
Run: 06, Epoch: 71, Loss: 0.4976, Train: 85.35%, Valid: 65.84% Test: 64.69%
Run: 06, Epoch: 72, Loss: 0.4739, Train: 85.71%, Valid: 65.29% Test: 63.82%
Run: 06, Epoch: 73, Loss: 0.4899, Train: 86.17%, Valid: 65.84% Test: 63.16%
Run: 06, Epoch: 74, Loss: 0.4682, Train: 87.00%, Valid: 66.80% Test: 64.47%
Run: 06, Epo

Run: 07, Epoch: 80, Loss: 0.3661, Train: 88.64%, Valid: 66.80% Test: 64.04%
Run: 07, Epoch: 81, Loss: 0.3740, Train: 89.01%, Valid: 66.39% Test: 62.72%
Run: 07, Epoch: 82, Loss: 0.3923, Train: 88.92%, Valid: 67.35% Test: 64.69%
Run: 07, Epoch: 83, Loss: 0.3944, Train: 88.37%, Valid: 67.76% Test: 64.25%
Run: 07, Epoch: 84, Loss: 0.3787, Train: 88.64%, Valid: 68.72% Test: 64.25%
Run: 07, Epoch: 85, Loss: 0.3654, Train: 89.29%, Valid: 67.22% Test: 64.47%
Run: 07, Epoch: 86, Loss: 0.3778, Train: 89.38%, Valid: 66.67% Test: 64.47%
Run: 07, Epoch: 87, Loss: 0.3693, Train: 89.29%, Valid: 66.94% Test: 64.47%
Run: 07, Epoch: 88, Loss: 0.3678, Train: 89.74%, Valid: 66.53% Test: 64.91%
Run: 07, Epoch: 89, Loss: 0.3663, Train: 88.83%, Valid: 66.53% Test: 64.04%
Run: 07, Epoch: 90, Loss: 0.3750, Train: 88.83%, Valid: 66.53% Test: 63.38%
Run: 07, Epoch: 91, Loss: 0.3692, Train: 89.01%, Valid: 65.84% Test: 62.72%
Run: 07, Epoch: 92, Loss: 0.3881, Train: 86.72%, Valid: 65.84% Test: 63.16%
Run: 07, Epo

Run: 08, Epoch: 99, Loss: 0.3666, Train: 90.11%, Valid: 64.20% Test: 64.04%
Run: 08, Epoch: 100, Loss: 0.3802, Train: 89.01%, Valid: 63.37% Test: 64.25%
Run 08:
Highest Train: 90.57
Highest Valid: 64.47
  Final Train: 90.57
   Final Test: 64.69
Run: 09, Epoch: 01, Loss: 1.9826, Train: 28.85%, Valid: 27.57% Test: 27.63%
Run: 09, Epoch: 02, Loss: 1.3683, Train: 28.75%, Valid: 27.43% Test: 27.63%
Run: 09, Epoch: 03, Loss: 1.2665, Train: 32.33%, Valid: 29.90% Test: 30.26%
Run: 09, Epoch: 04, Loss: 1.2287, Train: 36.08%, Valid: 31.41% Test: 32.89%
Run: 09, Epoch: 05, Loss: 1.1871, Train: 39.01%, Valid: 33.47% Test: 34.21%
Run: 09, Epoch: 06, Loss: 1.1745, Train: 45.70%, Valid: 37.04% Test: 39.47%
Run: 09, Epoch: 07, Loss: 1.1436, Train: 48.63%, Valid: 39.09% Test: 41.01%
Run: 09, Epoch: 08, Loss: 1.1330, Train: 49.73%, Valid: 40.74% Test: 43.42%
Run: 09, Epoch: 09, Loss: 1.1178, Train: 52.01%, Valid: 42.52% Test: 44.96%
Run: 09, Epoch: 10, Loss: 1.0934, Train: 53.75%, Valid: 44.99% Test: 45

Run: 10, Epoch: 18, Loss: 1.0155, Train: 61.26%, Valid: 52.67% Test: 56.14%
Run: 10, Epoch: 19, Loss: 1.0136, Train: 62.82%, Valid: 53.77% Test: 56.36%
Run: 10, Epoch: 20, Loss: 0.9869, Train: 64.29%, Valid: 55.42% Test: 57.46%
Run: 10, Epoch: 21, Loss: 0.9783, Train: 65.11%, Valid: 55.69% Test: 56.80%
Run: 10, Epoch: 22, Loss: 0.9591, Train: 65.66%, Valid: 56.10% Test: 57.24%
Run: 10, Epoch: 23, Loss: 0.9451, Train: 67.12%, Valid: 56.24% Test: 57.68%
Run: 10, Epoch: 24, Loss: 0.9207, Train: 67.22%, Valid: 56.52% Test: 58.55%
Run: 10, Epoch: 25, Loss: 0.8996, Train: 67.12%, Valid: 57.20% Test: 58.11%
Run: 10, Epoch: 26, Loss: 0.9061, Train: 67.95%, Valid: 58.57% Test: 57.46%
Run: 10, Epoch: 27, Loss: 0.8801, Train: 69.05%, Valid: 59.12% Test: 57.68%
Run: 10, Epoch: 28, Loss: 0.8753, Train: 69.14%, Valid: 58.30% Test: 58.77%
Run: 10, Epoch: 29, Loss: 0.8424, Train: 69.14%, Valid: 57.89% Test: 58.99%
Run: 10, Epoch: 30, Loss: 0.8400, Train: 70.97%, Valid: 59.81% Test: 61.40%
Run: 10, Epo

# Wise Embedding

In [8]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [9]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,2316,2317,2318,2319,2320,2321,2322,2323,2324,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [10]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.01)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [13]:
def Similarity(array1, array2):
    intersection = np.sum(np.logical_and(array1, array2))
    #union = np.sum(np.logical_or(array1, array2))
    #jaccard_similarity = intersection / union
    
    #return jaccard_similarity
    return intersection

In [11]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

[[3.0, 3.0, 3.0, 2.0, 3.0], [6.0, 5.0, 8.0, 10.0, 9.0], [9.0, 13.0, 22.0, 23.0, 21.0], [7.0, 10.0, 11.0, 14.0, 14.0], [16.0, 23.0, 41.0, 37.0, 37.0], [3.0, 2.0, 3.0, 3.0, 3.0], [1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0], [4.0, 5.0, 6.0, 6.0, 6.0], [1.0, 3.0, 3.0, 3.0, 3.0], [0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 2.0, 2.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0], [5.0, 8.0, 11.0, 11.0, 20.0], [10.0, 10.0, 13.0, 14.0, 16.0], [10.0, 10.0, 14.0, 16.0, 20.0], [1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0], [2.0, 2.0, 4.0, 14.0, 4.0], [1.0, 1.0, 0.0, 1.0, 1.0], [2.0, 2.0, 3.0, 3.0, 3.0]]


In [14]:

Fec=[]
for i in range(Number_nodes):
#for i in range(2):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    vec.append(Similarity(f,basis[0]))
    vec.append(Similarity(f,basis[1]))
    vec.append(Similarity(f,basis[2]))
    vec.append(Similarity(f,basis[3]))
    vec.append(Similarity(f,basis[4]))
    #print(f)
    f.clear()
    Fec.append(vec)
SFec=[]
for i in range(Number_nodes):
#for i in range(2):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    Svec.append(Similarity(f,sel_basis[0]))
    Svec.append(Similarity(f,sel_basis[1]))
    Svec.append(Similarity(f,sel_basis[2]))
    Svec.append(Similarity(f,sel_basis[3]))
    Svec.append(Similarity(f,sel_basis[4]))
    #print(f)
    f.clear()
    SFec.append(Svec)

In [ ]:
print(Fec)

In [ ]:
print(SFec)

In [15]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[ 7.,  6.,  6.,  ...,  5.,  5.,  7.],
        [10., 12., 18.,  ...,  8., 16., 15.],
        [19., 24., 39.,  ..., 19., 33., 28.],
        ...,
        [ 3.,  2.,  1.,  ...,  0.,  2.,  2.],
        [16., 20., 32.,  ..., 17., 22., 38.],
        [ 4.,  4.,  3.,  ...,  2.,  1.,  2.]])


'torch.FloatTensor'

# W-GCN

In [16]:
data.x=CC_domain
print(data)

Data(x=[2277, 10], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


In [17]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 48, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.05}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0673, Train: 17.22%, Valid: 15.50% Test: 19.08%
Run: 01, Epoch: 02, Loss: 1.4177, Train: 31.68%, Valid: 30.59% Test: 31.36%
Run: 01, Epoch: 03, Loss: 1.4314, Train: 22.62%, Valid: 20.44% Test: 23.90%
Run: 01, Epoch: 04, Loss: 1.4224, Train: 36.81%, Valid: 34.29% Test: 35.53%
Run: 01, Epoch: 05, Loss: 1.4006, Train: 31.23%, Valid: 34.84% Test: 32.68%
Run: 01, Epoch: 06, Loss: 1.3768, Train: 36.08%, Valid: 39.51% Test: 39.04%
Run: 01, Epoch: 07, Loss: 1.3191, Train: 36.72%, Valid: 39.23% Test: 35.53%
Run: 01, Epoch: 08, Loss: 1.3918, Train: 39.93%, Valid: 40.88% Test: 39.47%
Run: 01, Epoch: 09, Loss: 1.2741, Train: 35.62%, Valid: 32.65% Test: 35.53%
Run: 01, Epoch: 10, Loss: 1.3451, Train: 38.46%, Valid: 34.43% Test: 36.62%
Run: 01, Epoch: 11, Loss: 1.3006, Train: 36.26%, Valid: 36.08% Test: 36.62%
Run: 01, Epoch: 12, Loss: 1.2950, Train: 34.71%, Valid: 34.29% Test: 34.43%
Run: 01, Epoch: 13, Loss: 1.3490, Train: 43.68%, Valid: 43.48% Test: 41.89%
Run: 01, Epo

Run: 01, Epoch: 131, Loss: 1.0864, Train: 58.97%, Valid: 58.85% Test: 56.80%
Run: 01, Epoch: 132, Loss: 1.0654, Train: 57.88%, Valid: 58.02% Test: 57.02%
Run: 01, Epoch: 133, Loss: 1.0585, Train: 59.07%, Valid: 57.75% Test: 58.11%
Run: 01, Epoch: 134, Loss: 1.0508, Train: 59.89%, Valid: 58.98% Test: 59.65%
Run: 01, Epoch: 135, Loss: 1.0651, Train: 60.35%, Valid: 59.67% Test: 57.02%
Run: 01, Epoch: 136, Loss: 1.0444, Train: 63.74%, Valid: 61.45% Test: 60.96%
Run: 01, Epoch: 137, Loss: 1.0474, Train: 62.27%, Valid: 59.67% Test: 60.09%
Run: 01, Epoch: 138, Loss: 1.0728, Train: 62.73%, Valid: 60.22% Test: 62.28%
Run: 01, Epoch: 139, Loss: 1.0590, Train: 58.06%, Valid: 58.44% Test: 53.29%
Run: 01, Epoch: 140, Loss: 1.0556, Train: 62.36%, Valid: 61.04% Test: 58.77%
Run: 01, Epoch: 141, Loss: 1.0642, Train: 62.09%, Valid: 59.12% Test: 60.75%
Run: 01, Epoch: 142, Loss: 1.0502, Train: 61.54%, Valid: 58.30% Test: 60.09%
Run: 01, Epoch: 143, Loss: 1.0485, Train: 57.14%, Valid: 55.28% Test: 53.51%

Run: 02, Epoch: 43, Loss: 1.1767, Train: 54.76%, Valid: 50.07% Test: 57.24%
Run: 02, Epoch: 44, Loss: 1.1896, Train: 49.18%, Valid: 45.27% Test: 52.63%
Run: 02, Epoch: 45, Loss: 1.1775, Train: 47.80%, Valid: 44.17% Test: 49.78%
Run: 02, Epoch: 46, Loss: 1.1823, Train: 49.36%, Valid: 47.33% Test: 50.22%
Run: 02, Epoch: 47, Loss: 1.1772, Train: 52.01%, Valid: 51.71% Test: 54.82%
Run: 02, Epoch: 48, Loss: 1.1418, Train: 53.75%, Valid: 52.26% Test: 56.58%
Run: 02, Epoch: 49, Loss: 1.1504, Train: 52.93%, Valid: 52.26% Test: 53.73%
Run: 02, Epoch: 50, Loss: 1.1349, Train: 54.49%, Valid: 52.54% Test: 57.46%
Run: 02, Epoch: 51, Loss: 1.1279, Train: 51.47%, Valid: 49.66% Test: 55.70%
Run: 02, Epoch: 52, Loss: 1.1534, Train: 51.28%, Valid: 49.25% Test: 51.75%
Run: 02, Epoch: 53, Loss: 1.1882, Train: 54.85%, Valid: 51.71% Test: 57.89%
Run: 02, Epoch: 54, Loss: 1.1633, Train: 55.49%, Valid: 52.54% Test: 57.24%
Run: 02, Epoch: 55, Loss: 1.1595, Train: 50.18%, Valid: 46.91% Test: 50.88%
Run: 02, Epo

Run: 02, Epoch: 161, Loss: 1.0540, Train: 59.16%, Valid: 57.20% Test: 59.65%
Run: 02, Epoch: 162, Loss: 1.0586, Train: 57.78%, Valid: 56.52% Test: 62.94%
Run: 02, Epoch: 163, Loss: 1.0374, Train: 55.59%, Valid: 55.28% Test: 62.06%
Run: 02, Epoch: 164, Loss: 1.0711, Train: 58.70%, Valid: 56.24% Test: 61.62%
Run: 02, Epoch: 165, Loss: 1.0506, Train: 60.16%, Valid: 55.14% Test: 59.21%
Run: 02, Epoch: 166, Loss: 1.0646, Train: 60.44%, Valid: 55.28% Test: 60.75%
Run: 02, Epoch: 167, Loss: 1.0341, Train: 55.40%, Valid: 54.60% Test: 57.24%
Run: 02, Epoch: 168, Loss: 1.0670, Train: 59.52%, Valid: 59.26% Test: 63.38%
Run: 02, Epoch: 169, Loss: 1.0459, Train: 60.99%, Valid: 57.61% Test: 63.38%
Run: 02, Epoch: 170, Loss: 1.0491, Train: 57.88%, Valid: 55.83% Test: 61.62%
Run: 02, Epoch: 171, Loss: 1.0284, Train: 55.59%, Valid: 54.32% Test: 57.68%
Run: 02, Epoch: 172, Loss: 1.0296, Train: 56.59%, Valid: 53.36% Test: 54.61%
Run: 02, Epoch: 173, Loss: 1.0764, Train: 59.71%, Valid: 56.65% Test: 61.18%

Run: 03, Epoch: 76, Loss: 1.1140, Train: 58.15%, Valid: 56.65% Test: 51.97%
Run: 03, Epoch: 77, Loss: 1.0984, Train: 55.86%, Valid: 51.71% Test: 51.75%
Run: 03, Epoch: 78, Loss: 1.1264, Train: 54.76%, Valid: 52.67% Test: 52.85%
Run: 03, Epoch: 79, Loss: 1.1198, Train: 55.77%, Valid: 53.91% Test: 51.97%
Run: 03, Epoch: 80, Loss: 1.0805, Train: 56.50%, Valid: 54.87% Test: 52.19%
Run: 03, Epoch: 81, Loss: 1.0948, Train: 55.49%, Valid: 53.50% Test: 50.88%
Run: 03, Epoch: 82, Loss: 1.0901, Train: 54.03%, Valid: 51.99% Test: 50.22%
Run: 03, Epoch: 83, Loss: 1.0902, Train: 52.11%, Valid: 49.66% Test: 49.78%
Run: 03, Epoch: 84, Loss: 1.0984, Train: 52.66%, Valid: 49.38% Test: 49.12%
Run: 03, Epoch: 85, Loss: 1.0956, Train: 51.74%, Valid: 49.38% Test: 49.34%
Run: 03, Epoch: 86, Loss: 1.0945, Train: 54.58%, Valid: 51.17% Test: 51.54%
Run: 03, Epoch: 87, Loss: 1.0817, Train: 55.68%, Valid: 51.17% Test: 52.63%
Run: 03, Epoch: 88, Loss: 1.0861, Train: 53.30%, Valid: 49.93% Test: 48.90%
Run: 03, Epo

...

Run: 08, Epoch: 111, Loss: 1.0544, Train: 58.97%, Valid: 58.16% Test: 58.77%
Run: 08, Epoch: 112, Loss: 1.0734, Train: 58.33%, Valid: 55.56% Test: 55.48%
Run: 08, Epoch: 113, Loss: 1.0430, Train: 59.52%, Valid: 56.24% Test: 55.92%
Run: 08, Epoch: 114, Loss: 1.0249, Train: 61.72%, Valid: 58.16% Test: 58.55%
Run: 08, Epoch: 115, Loss: 1.0307, Train: 54.49%, Valid: 54.05% Test: 53.95%
Run: 08, Epoch: 116, Loss: 1.0711, Train: 57.05%, Valid: 55.01% Test: 52.85%
Run: 08, Epoch: 117, Loss: 1.0354, Train: 58.24%, Valid: 54.60% Test: 50.00%
Run: 08, Epoch: 118, Loss: 1.0467, Train: 60.16%, Valid: 58.44% Test: 56.58%
Run: 08, Epoch: 119, Loss: 1.0195, Train: 61.63%, Valid: 57.75% Test: 59.21%
Run: 08, Epoch: 120, Loss: 1.0251, Train: 57.23%, Valid: 54.32% Test: 57.24%
Run: 08, Epoch: 121, Loss: 1.0285, Train: 60.35%, Valid: 57.06% Test: 57.46%
Run: 08, Epoch: 122, Loss: 0.9976, Train: 62.82%, Valid: 58.98% Test: 57.46%
Run: 08, Epoch: 123, Loss: 1.0000, Train: 62.36%, Valid: 57.34% Test: 56.80%

Run: 09, Epoch: 24, Loss: 1.2066, Train: 45.70%, Valid: 46.50% Test: 48.25%
Run: 09, Epoch: 25, Loss: 1.1966, Train: 42.95%, Valid: 42.25% Test: 43.42%
Run: 09, Epoch: 26, Loss: 1.2083, Train: 38.83%, Valid: 38.68% Test: 38.38%
Run: 09, Epoch: 27, Loss: 1.1858, Train: 43.96%, Valid: 43.21% Test: 44.96%
Run: 09, Epoch: 28, Loss: 1.1776, Train: 45.33%, Valid: 45.40% Test: 47.15%
Run: 09, Epoch: 29, Loss: 1.1997, Train: 49.18%, Valid: 50.34% Test: 50.22%
Run: 09, Epoch: 30, Loss: 1.1648, Train: 46.52%, Valid: 46.23% Test: 47.15%
Run: 09, Epoch: 31, Loss: 1.1775, Train: 47.99%, Valid: 49.66% Test: 49.56%
Run: 09, Epoch: 32, Loss: 1.1592, Train: 49.91%, Valid: 49.11% Test: 51.10%
Run: 09, Epoch: 33, Loss: 1.1562, Train: 49.18%, Valid: 47.74% Test: 51.10%
Run: 09, Epoch: 34, Loss: 1.1646, Train: 49.73%, Valid: 48.42% Test: 50.22%
Run: 09, Epoch: 35, Loss: 1.1659, Train: 49.08%, Valid: 51.44% Test: 50.66%
Run: 09, Epoch: 36, Loss: 1.1278, Train: 49.63%, Valid: 51.30% Test: 50.22%
Run: 09, Epo

Run: 09, Epoch: 138, Loss: 1.0718, Train: 57.69%, Valid: 58.16% Test: 59.43%
Run: 09, Epoch: 139, Loss: 1.0385, Train: 55.95%, Valid: 55.01% Test: 56.36%
Run: 09, Epoch: 140, Loss: 1.0652, Train: 57.33%, Valid: 59.26% Test: 57.46%
Run: 09, Epoch: 141, Loss: 1.0460, Train: 55.59%, Valid: 55.83% Test: 57.68%
Run: 09, Epoch: 142, Loss: 1.0686, Train: 59.89%, Valid: 59.12% Test: 60.09%
Run: 09, Epoch: 143, Loss: 1.0470, Train: 58.70%, Valid: 58.44% Test: 59.65%
Run: 09, Epoch: 144, Loss: 1.0247, Train: 59.62%, Valid: 59.81% Test: 60.53%
Run: 09, Epoch: 145, Loss: 1.0452, Train: 60.07%, Valid: 58.30% Test: 58.55%
Run: 09, Epoch: 146, Loss: 1.0555, Train: 61.26%, Valid: 61.73% Test: 60.75%
Run: 09, Epoch: 147, Loss: 1.0421, Train: 61.08%, Valid: 60.36% Test: 60.75%
Run: 09, Epoch: 148, Loss: 1.0323, Train: 58.06%, Valid: 57.34% Test: 60.31%
Run: 09, Epoch: 149, Loss: 1.0480, Train: 61.08%, Valid: 62.00% Test: 65.13%
Run: 09, Epoch: 150, Loss: 1.0272, Train: 59.89%, Valid: 60.08% Test: 64.04%

Run: 10, Epoch: 49, Loss: 1.1492, Train: 53.75%, Valid: 51.99% Test: 53.51%
Run: 10, Epoch: 50, Loss: 1.1428, Train: 53.39%, Valid: 51.71% Test: 51.54%
Run: 10, Epoch: 51, Loss: 1.1427, Train: 56.14%, Valid: 53.64% Test: 52.85%
Run: 10, Epoch: 52, Loss: 1.1242, Train: 55.49%, Valid: 54.87% Test: 54.82%
Run: 10, Epoch: 53, Loss: 1.1213, Train: 53.57%, Valid: 51.30% Test: 54.17%
Run: 10, Epoch: 54, Loss: 1.1247, Train: 50.55%, Valid: 48.83% Test: 50.00%
Run: 10, Epoch: 55, Loss: 1.1260, Train: 50.09%, Valid: 50.34% Test: 50.66%
Run: 10, Epoch: 56, Loss: 1.1283, Train: 50.55%, Valid: 52.26% Test: 51.75%
Run: 10, Epoch: 57, Loss: 1.1309, Train: 58.42%, Valid: 54.73% Test: 57.24%
Run: 10, Epoch: 58, Loss: 1.1003, Train: 59.98%, Valid: 55.69% Test: 58.33%
Run: 10, Epoch: 59, Loss: 1.0968, Train: 59.25%, Valid: 56.24% Test: 57.02%
Run: 10, Epoch: 60, Loss: 1.1054, Train: 58.61%, Valid: 55.97% Test: 56.58%
Run: 10, Epoch: 61, Loss: 1.1115, Train: 55.04%, Valid: 50.34% Test: 53.51%
Run: 10, Epo

Run: 10, Epoch: 167, Loss: 1.0170, Train: 58.79%, Valid: 55.56% Test: 60.09%
Run: 10, Epoch: 168, Loss: 0.9905, Train: 60.53%, Valid: 56.38% Test: 58.33%
Run: 10, Epoch: 169, Loss: 1.0069, Train: 58.70%, Valid: 55.14% Test: 57.89%
Run: 10, Epoch: 170, Loss: 0.9927, Train: 59.25%, Valid: 56.52% Test: 58.99%
Run: 10, Epoch: 171, Loss: 0.9882, Train: 62.73%, Valid: 58.98% Test: 61.40%
Run: 10, Epoch: 172, Loss: 0.9932, Train: 62.00%, Valid: 58.30% Test: 59.65%
Run: 10, Epoch: 173, Loss: 1.0039, Train: 63.37%, Valid: 61.59% Test: 62.06%
Run: 10, Epoch: 174, Loss: 0.9886, Train: 60.16%, Valid: 58.02% Test: 58.33%
Run: 10, Epoch: 175, Loss: 0.9874, Train: 59.89%, Valid: 57.61% Test: 59.65%
Run: 10, Epoch: 176, Loss: 1.0167, Train: 62.45%, Valid: 59.40% Test: 60.96%
Run: 10, Epoch: 177, Loss: 0.9519, Train: 61.72%, Valid: 58.44% Test: 57.68%
Run: 10, Epoch: 178, Loss: 0.9911, Train: 62.36%, Valid: 58.85% Test: 60.96%
Run: 10, Epoch: 179, Loss: 0.9710, Train: 63.74%, Valid: 61.59% Test: 64.91%

# Topological Encoddnig 

In [18]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon')
data = dataset[0]
print(data)

Data(x=[2277, 2325], edge_index=[2, 36101], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10])


In [ ]:
print(data.edge_index.numpy())

In [19]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [ ]:
# a "plain" graph is undirected
#G = nx.DiGraph()
G = nx.Graph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [ ]:
print(G.number_of_edges())

In [ ]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [ ]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [ ]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

In [ ]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,12,15,20,25,30,50,100,200,400]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

In [6]:
data = pd.read_csv('Feature_Cham.csv')
data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1
0,0,0,5,5,13,14,14,14,8,8,...,3,5,13,13,10,10,10,10,10,10
1,1,4,13,20,25,27,24,25,27,26,...,0,0,3,13,40,56,25,3,3,3
2,2,3,4,11,12,17,18,26,26,30,...,0,4,6,13,23,38,170,308,44,13
3,3,7,7,7,7,7,7,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4,33,89,109,134,201,266,309,324,328,...,14,15,27,53,105,185,1191,1169,617,617


In [7]:
Data1=data.drop(['Unnamed: 0'], axis=1)
Data1.head()

,0,1,2,3,4,5,6,7,8,9,...,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1
0,0,5,5,13,14,14,14,8,8,8,...,3,5,13,13,10,10,10,10,10,10
1,4,13,20,25,27,24,25,27,26,27,...,0,0,3,13,40,56,25,3,3,3
2,3,4,11,12,17,18,26,26,30,31,...,0,4,6,13,23,38,170,308,44,13
3,7,7,7,7,7,7,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,33,89,109,134,201,266,309,324,328,321,...,14,15,27,53,105,185,1191,1169,617,617


In [8]:
Topo_fe=torch.tensor(Data1.values).float()

In [9]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101])


# Topo-W-GCN

In [10]:
data.x=CC_domain
data.topo=Topo_fe
print(data)

Data(x=[2277, 2325], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10], adj_t=[2277, 2277, nnz=36101], topo=[2277, 38])


In [ ]:
print(len(topo_fe[0]))

In [11]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [27]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 48, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.01)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.6227, Train: 25.64%, Valid: 26.06% Test: 25.44%
Run: 01, Epoch: 02, Loss: 1.4579, Train: 27.38%, Valid: 27.98% Test: 28.07%
Run: 01, Epoch: 03, Loss: 1.4099, Train: 22.07%, Valid: 24.42% Test: 22.37%
Run: 01, Epoch: 04, Loss: 1.3745, Train: 22.07%, Valid: 24.42% Test: 22.37%
Run: 01, Epoch: 05, Loss: 1.3475, Train: 22.07%, Valid: 24.55% Test: 22.81%
Run: 01, Epoch: 06, Loss: 1.3361, Train: 23.81%, Valid: 27.30% Test: 23.90%
Run: 01, Epoch: 07, Loss: 1.3061, Train: 25.27%, Valid: 27.02% Test: 25.00%
Run: 01, Epoch: 08, Loss: 1.2949, Train: 33.61%, Valid: 36.35% Test: 35.31%
Run: 01, Epoch: 09, Loss: 1.2558, Train: 37.09%, Valid: 41.84% Test: 41.67%
Run: 01, Epoch: 10, Loss: 1.2335, Train: 39.84%, Valid: 42.39% Test: 41.01%
Run: 01, Epoch: 11, Loss: 1.2216, Train: 37.45%, Valid: 38.96% Test: 37.50%
Run: 01, Epoch: 12, Loss: 1.2218, Train: 38.19%, Valid: 38.68% Test: 36.62%
Run: 01, Epoch: 13, Loss: 1.2245, Train: 40.38%, Valid: 39.92% Test: 39.69%
Run: 01, Epo

Run: 02, Epoch: 07, Loss: 1.2850, Train: 27.75%, Valid: 27.16% Test: 30.48%
Run: 02, Epoch: 08, Loss: 1.2678, Train: 30.86%, Valid: 29.90% Test: 33.77%
Run: 02, Epoch: 09, Loss: 1.2471, Train: 31.68%, Valid: 31.28% Test: 35.75%
Run: 02, Epoch: 10, Loss: 1.2276, Train: 44.78%, Valid: 42.25% Test: 46.27%
Run: 02, Epoch: 11, Loss: 1.1842, Train: 44.69%, Valid: 42.94% Test: 46.05%
Run: 02, Epoch: 12, Loss: 1.1750, Train: 45.42%, Valid: 43.90% Test: 46.05%
Run: 02, Epoch: 13, Loss: 1.1723, Train: 44.87%, Valid: 42.66% Test: 45.18%
Run: 02, Epoch: 14, Loss: 1.1475, Train: 53.02%, Valid: 50.07% Test: 51.75%
Run: 02, Epoch: 15, Loss: 1.1651, Train: 46.52%, Valid: 46.36% Test: 50.88%
Run: 02, Epoch: 16, Loss: 1.1434, Train: 56.23%, Valid: 53.91% Test: 59.65%
Run: 02, Epoch: 17, Loss: 1.1430, Train: 55.95%, Valid: 51.99% Test: 58.11%
Run: 02, Epoch: 18, Loss: 1.1153, Train: 50.46%, Valid: 48.01% Test: 52.85%
Run: 02, Epoch: 19, Loss: 1.1274, Train: 57.05%, Valid: 52.67% Test: 57.89%
Run: 02, Epo

Run: 03, Epoch: 21, Loss: 1.1216, Train: 49.63%, Valid: 47.46% Test: 46.27%
Run: 03, Epoch: 22, Loss: 1.1348, Train: 46.70%, Valid: 46.78% Test: 46.71%
Run: 03, Epoch: 23, Loss: 1.1481, Train: 46.34%, Valid: 45.13% Test: 43.20%
Run: 03, Epoch: 24, Loss: 1.1197, Train: 51.10%, Valid: 48.56% Test: 46.05%
Run: 03, Epoch: 25, Loss: 1.1244, Train: 52.47%, Valid: 48.01% Test: 47.15%
Run: 03, Epoch: 26, Loss: 1.1045, Train: 46.98%, Valid: 45.54% Test: 43.42%
Run: 03, Epoch: 27, Loss: 1.0851, Train: 47.44%, Valid: 45.54% Test: 44.52%
Run: 03, Epoch: 28, Loss: 1.0774, Train: 48.90%, Valid: 46.78% Test: 46.05%
Run: 03, Epoch: 29, Loss: 1.0819, Train: 50.09%, Valid: 47.05% Test: 47.81%
Run: 03, Epoch: 30, Loss: 1.0923, Train: 50.64%, Valid: 48.83% Test: 48.03%
Run: 03, Epoch: 31, Loss: 1.0756, Train: 54.03%, Valid: 51.58% Test: 47.37%
Run: 03, Epoch: 32, Loss: 1.0708, Train: 52.93%, Valid: 49.38% Test: 47.15%
Run: 03, Epoch: 33, Loss: 1.0466, Train: 52.38%, Valid: 48.42% Test: 46.93%
Run: 03, Epo

Run: 04, Epoch: 32, Loss: 1.0641, Train: 58.15%, Valid: 56.38% Test: 53.95%
Run: 04, Epoch: 33, Loss: 1.0670, Train: 59.16%, Valid: 56.24% Test: 54.61%
Run: 04, Epoch: 34, Loss: 1.0795, Train: 57.33%, Valid: 56.10% Test: 50.66%
Run: 04, Epoch: 35, Loss: 1.0619, Train: 52.56%, Valid: 49.79% Test: 49.78%
Run: 04, Epoch: 36, Loss: 1.0590, Train: 56.41%, Valid: 54.18% Test: 51.97%
Run: 04, Epoch: 37, Loss: 1.0374, Train: 54.58%, Valid: 53.91% Test: 52.41%
Run: 04, Epoch: 38, Loss: 1.0726, Train: 58.70%, Valid: 59.26% Test: 56.36%
Run: 04, Epoch: 39, Loss: 1.0793, Train: 55.13%, Valid: 52.13% Test: 54.61%
Run: 04, Epoch: 40, Loss: 1.0323, Train: 56.59%, Valid: 54.73% Test: 56.14%
Run: 04, Epoch: 41, Loss: 1.0495, Train: 62.18%, Valid: 58.71% Test: 57.24%
Run: 04, Epoch: 42, Loss: 1.0242, Train: 60.16%, Valid: 60.08% Test: 57.46%
Run: 04, Epoch: 43, Loss: 1.0183, Train: 60.26%, Valid: 58.44% Test: 55.04%
Run: 04, Epoch: 44, Loss: 1.0335, Train: 61.63%, Valid: 60.91% Test: 58.55%
Run: 04, Epo

Run: 05, Epoch: 45, Loss: 1.0254, Train: 59.43%, Valid: 56.79% Test: 58.11%
Run: 05, Epoch: 46, Loss: 1.0283, Train: 59.34%, Valid: 59.26% Test: 59.43%
Run: 05, Epoch: 47, Loss: 1.0321, Train: 61.45%, Valid: 59.53% Test: 60.53%
Run: 05, Epoch: 48, Loss: 1.0118, Train: 62.45%, Valid: 61.87% Test: 61.84%
Run: 05, Epoch: 49, Loss: 1.0243, Train: 60.62%, Valid: 58.71% Test: 59.43%
Run: 05, Epoch: 50, Loss: 1.0166, Train: 58.52%, Valid: 57.48% Test: 58.33%
Run: 05, Epoch: 51, Loss: 1.0064, Train: 60.16%, Valid: 60.08% Test: 61.18%
Run: 05, Epoch: 52, Loss: 1.0236, Train: 60.07%, Valid: 59.81% Test: 61.40%
Run: 05, Epoch: 53, Loss: 1.0027, Train: 59.71%, Valid: 58.98% Test: 60.53%
Run: 05, Epoch: 54, Loss: 1.0245, Train: 61.17%, Valid: 60.91% Test: 62.50%
Run: 05, Epoch: 55, Loss: 1.0012, Train: 61.17%, Valid: 59.53% Test: 61.84%
Run: 05, Epoch: 56, Loss: 0.9906, Train: 58.97%, Valid: 55.28% Test: 56.36%
Run: 05, Epoch: 57, Loss: 1.0098, Train: 59.43%, Valid: 55.97% Test: 57.68%
Run: 05, Epo

Run: 06, Epoch: 57, Loss: 0.9695, Train: 62.64%, Valid: 59.81% Test: 59.87%
Run: 06, Epoch: 58, Loss: 0.9700, Train: 56.87%, Valid: 55.01% Test: 53.73%
Run: 06, Epoch: 59, Loss: 1.0071, Train: 57.60%, Valid: 56.93% Test: 57.46%
Run: 06, Epoch: 60, Loss: 0.9686, Train: 56.14%, Valid: 56.38% Test: 56.80%
Run: 06, Epoch: 61, Loss: 0.9918, Train: 54.58%, Valid: 54.60% Test: 55.92%
Run: 06, Epoch: 62, Loss: 0.9554, Train: 56.32%, Valid: 53.50% Test: 55.70%
Run: 06, Epoch: 63, Loss: 0.9883, Train: 58.97%, Valid: 55.42% Test: 58.11%
Run: 06, Epoch: 64, Loss: 0.9755, Train: 64.29%, Valid: 61.45% Test: 59.21%
Run: 06, Epoch: 65, Loss: 0.9897, Train: 65.11%, Valid: 62.14% Test: 59.43%
Run: 06, Epoch: 66, Loss: 0.9923, Train: 62.36%, Valid: 60.08% Test: 59.21%
Run: 06, Epoch: 67, Loss: 0.9415, Train: 61.72%, Valid: 60.91% Test: 60.75%
Run: 06, Epoch: 68, Loss: 0.9995, Train: 65.38%, Valid: 62.00% Test: 60.75%
Run: 06, Epoch: 69, Loss: 0.9614, Train: 65.20%, Valid: 61.59% Test: 59.65%
Run: 06, Epo

Run: 07, Epoch: 72, Loss: 0.8841, Train: 60.44%, Valid: 58.98% Test: 55.70%
Run: 07, Epoch: 73, Loss: 0.8987, Train: 62.64%, Valid: 56.24% Test: 51.75%
Run: 07, Epoch: 74, Loss: 0.9094, Train: 62.45%, Valid: 58.30% Test: 53.95%
Run: 07, Epoch: 75, Loss: 0.9094, Train: 62.09%, Valid: 57.89% Test: 53.07%
Run: 07, Epoch: 76, Loss: 0.8934, Train: 64.38%, Valid: 60.36% Test: 56.58%
Run: 07, Epoch: 77, Loss: 0.9036, Train: 60.62%, Valid: 57.06% Test: 53.29%
Run: 07, Epoch: 78, Loss: 0.9091, Train: 67.86%, Valid: 62.83% Test: 57.02%
Run: 07, Epoch: 79, Loss: 0.8848, Train: 66.12%, Valid: 62.83% Test: 55.48%
Run: 07, Epoch: 80, Loss: 0.9032, Train: 65.38%, Valid: 60.63% Test: 54.61%
Run: 07, Epoch: 81, Loss: 0.8914, Train: 63.64%, Valid: 60.91% Test: 55.04%
Run: 07, Epoch: 82, Loss: 0.8827, Train: 67.95%, Valid: 64.06% Test: 55.70%
Run: 07, Epoch: 83, Loss: 0.8950, Train: 66.30%, Valid: 61.59% Test: 57.89%
Run: 07, Epoch: 84, Loss: 0.8868, Train: 69.14%, Valid: 62.55% Test: 58.77%
Run: 07, Epo

Run: 08, Epoch: 84, Loss: 0.9218, Train: 63.83%, Valid: 59.40% Test: 59.87%
Run: 08, Epoch: 85, Loss: 0.9196, Train: 64.74%, Valid: 60.63% Test: 57.68%
Run: 08, Epoch: 86, Loss: 0.9354, Train: 64.29%, Valid: 60.77% Test: 56.58%
Run: 08, Epoch: 87, Loss: 0.9243, Train: 63.83%, Valid: 61.18% Test: 57.89%
Run: 08, Epoch: 88, Loss: 0.9205, Train: 62.55%, Valid: 60.91% Test: 57.46%
Run: 08, Epoch: 89, Loss: 0.9183, Train: 62.00%, Valid: 59.12% Test: 56.36%
Run: 08, Epoch: 90, Loss: 0.8894, Train: 63.10%, Valid: 59.26% Test: 57.24%
Run: 08, Epoch: 91, Loss: 0.9303, Train: 61.90%, Valid: 57.61% Test: 55.92%
Run: 08, Epoch: 92, Loss: 0.9099, Train: 62.00%, Valid: 58.71% Test: 57.46%
Run: 08, Epoch: 93, Loss: 0.9143, Train: 62.09%, Valid: 58.44% Test: 56.58%
Run: 08, Epoch: 94, Loss: 0.8991, Train: 63.83%, Valid: 57.89% Test: 56.58%
Run: 08, Epoch: 95, Loss: 0.9151, Train: 64.56%, Valid: 57.75% Test: 55.70%
Run: 08, Epoch: 96, Loss: 0.9156, Train: 64.56%, Valid: 59.95% Test: 55.04%
Run: 08, Epo

Run: 09, Epoch: 92, Loss: 0.9449, Train: 61.72%, Valid: 62.00% Test: 63.16%
Run: 09, Epoch: 93, Loss: 0.9530, Train: 61.54%, Valid: 62.00% Test: 63.38%
Run: 09, Epoch: 94, Loss: 0.9249, Train: 61.26%, Valid: 61.04% Test: 62.94%
Run: 09, Epoch: 95, Loss: 0.9476, Train: 62.09%, Valid: 61.59% Test: 62.50%
Run: 09, Epoch: 96, Loss: 0.9427, Train: 61.08%, Valid: 60.36% Test: 61.40%
Run: 09, Epoch: 97, Loss: 0.9560, Train: 61.63%, Valid: 62.28% Test: 64.69%
Run: 09, Epoch: 98, Loss: 0.9397, Train: 62.00%, Valid: 63.24% Test: 64.91%
Run: 09, Epoch: 99, Loss: 0.9422, Train: 60.44%, Valid: 60.22% Test: 62.06%
Run: 09, Epoch: 100, Loss: 0.9464, Train: 63.00%, Valid: 62.28% Test: 66.89%
Run 09:
Highest Train: 65.29
Highest Valid: 65.71
  Final Train: 65.29
   Final Test: 66.23
Run: 10, Epoch: 01, Loss: 1.6417, Train: 24.91%, Valid: 27.16% Test: 30.04%
Run: 10, Epoch: 02, Loss: 1.4774, Train: 20.88%, Valid: 20.99% Test: 19.96%
Run: 10, Epoch: 03, Loss: 1.4117, Train: 29.49%, Valid: 29.22% Test: 30

# Topo-GCN

In [ ]:
dataset = WikipediaNetwork(root='/tmp/chameleon', name='chameleon',transform=T.ToSparseTensor())
data = dataset[0]
data.topo=Topo_fe
print(data)

In [13]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 48, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.0,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.01)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, ' f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                 #     f'Valid: {100 * valid_acc:.2f}% '
                  #    f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Run 01:
Highest Train: 93.96
Highest Valid: 69.82
  Final Train: 92.31
   Final Test: 65.35
Run 02:
Highest Train: 93.04
Highest Valid: 65.57
  Final Train: 90.57
   Final Test: 70.83
Run 03:
Highest Train: 94.41
Highest Valid: 68.72
  Final Train: 94.23
   Final Test: 67.76
Run 04:
Highest Train: 92.03
Highest Valid: 68.31
  Final Train: 89.84
   Final Test: 66.01
Run 05:
Highest Train: 93.77
Highest Valid: 68.45
  Final Train: 92.86
   Final Test: 67.76
Run 06:
Highest Train: 93.68
Highest Valid: 67.63
  Final Train: 90.11
   Final Test: 67.11
Run 07:
Highest Train: 93.32
Highest Valid: 68.45
  Final Train: 88.28
   Final Test: 64.69
Run 08:
Highest Train: 93.50
Highest Valid: 65.43
  Final Train: 90.66
   Final Test: 64.47
Run 09:
Highest Train: 91.48
Highest Valid: 69.14
  Final Train: 90.02
   Final Test: 67.11
Run 10:
Highest Train: 94.05
Highest Valid: 67.76
  Final Train: 94.05
   Final Test: 68.20
All runs:
Highest Train: 93.32 ± 0.92
Highest Valid: 67.93 ± 1.43
  Final Train: